# VoxCity Demo — OBJ Export (Advanced & Simulation Results)

This notebook exports the voxel city and simulation results to OBJ, demonstrating value-mapped OBJ for analysis layers.


In [ ]:
# %pip install voxcity

from voxcity.generator import get_voxcity
from voxcity.exporter.obj import export_obj, grid_to_obj
from voxcity.simulator.solar import get_global_solar_irradiance_using_epw
from voxcity.simulator.view import get_view_index

meshsize = 5
rectangle_vertices = [
    (139.760, 35.680),
    (139.760, 35.690),
    (139.770, 35.690),
    (139.770, 35.680)
]

voxcity_grid, building_height_grid, building_min_height_grid, \
    building_id_grid, canopy_height_grid, canopy_bottom_height_grid, land_cover_grid, dem_grid, \
    building_gdf = get_voxcity(
        rectangle_vertices,
        'OpenStreetMap', 'OpenStreetMap',
        'High Resolution 1m Global Canopy Height Maps',
        'DeltaDTM',
        meshsize,
        output_dir='output/obj_demo'
)

print(voxcity_grid.shape, dem_grid.shape)


## Export voxel city as OBJ


In [ ]:
export_obj(voxcity_grid, output_dir='output/obj_demo', file_name='voxcity', voxel_size=meshsize)
print('Exported voxcity OBJ')


## Export simulation results as value-mapped OBJ
Examples: instantaneous solar irradiance, cumulative solar, and view indices.


In [ ]:
# Instantaneous solar
solar_kwargs = {
    "download_nearest_epw": True,
    "rectangle_vertices": rectangle_vertices,
    "calc_time": "01-01 12:00:00",
    "view_point_height": 1.5,
    "dem_grid": dem_grid,
}
solar_grid = get_global_solar_irradiance_using_epw(
    voxcity_grid, meshsize, calc_type='instantaneous', **solar_kwargs
)

# Export instantaneous solar as colored OBJ
from voxcity.exporter.obj import grid_to_obj

grid_to_obj(
    solar_grid, dem_grid,
    output_dir='output/obj_demo', file_name='solar_instantaneous',
    cell_size=meshsize, offset=1.5, colormap_name='magma', num_colors=10, alpha=1.0,
    vmin=0
)

# Cumulative solar for a time window
cum_kwargs = solar_kwargs.copy()
cum_kwargs["start_time"] = "01-01 05:00:00"
cum_kwargs["end_time"] = "01-31 20:00:00"

cum_solar_grid = get_global_solar_irradiance_using_epw(
    voxcity_grid, meshsize, calc_type='cumulative', **cum_kwargs
)

grid_to_obj(
    cum_solar_grid, dem_grid,
    output_dir='output/obj_demo', file_name='solar_cumulative',
    cell_size=meshsize, offset=1.5, colormap_name='viridis', num_colors=10, alpha=1.0
)

# View indices
gvi = get_view_index(voxcity_grid, meshsize, mode='green', obj_export=False)

grid_to_obj(
    gvi, dem_grid,
    output_dir='output/obj_demo', file_name='gvi',
    cell_size=meshsize, offset=1.5, colormap_name='Greens', num_colors=10, alpha=1.0,
    vmin=0.0, vmax=1.0
)
